In [1]:
%load_ext autoreload

In [6]:
%autoreload 2
import h5py
from matplotlib import colors
import matplotlib.pyplot as plt
import multiprocessing
from neural_analysis.matIO import loadmat
import numpy as np
import os
import pandas as pd
import ssm
from ssm.util import find_permutation
#Load from extensions
from ssm.extensions.mp_srslds.emissions_ext import GaussianOrthogonalCompoundEmissions
from ssm.extensions.mp_srslds.transitions_ext import StickyRecurrentOnlyTransitions
import sys
import time
from tqdm.auto import tqdm

sys.path.append('../..')
from multi_process import slds_eigs_worker
from nld_utils import simulate_lorenz
from up_down import get_up_down
from utils import get_binary_stimuli, get_sample_interval, load, save

## Load Neural Data

In [4]:
# filename = '../../__data__/Mary-Anesthesia-20160809-01.mat'
filename = r'/home/adameisen/millerdata/common/datasets/anesthesia/mat/propofolPuffTone/Mary-Anesthesia-20160809-01.mat'
# filename = r'/home/adameisen/common/datasets/anesthesia/mat/propofolWakeUp/Mary-Anesthesia-20170203-02.mat'
print("Loading data ...")
start = time.process_time()
electrode_info, lfp, lfp_schema, session_info, spike_times, unit_info = loadmat(filename, variables=['electrodeInfo', 'lfp', 'lfpSchema', 'sessionInfo', 'spikeTimes', 'unitInfo'], verbose=False)
# electrode_info, lfp_schema, session_info, unit_info = loadmat(filename, variables=['electrodeInfo', 'lfpSchema', 'sessionInfo', 'unitInfo'], verbose=False)
dt = lfp_schema['smpInterval'][0]

f = h5py.File(filename, 'r')
airPuff_binary, audio_binary = get_binary_stimuli(f)

print(f"Data loaded (took {time.process_time() - start:.2f} seconds)")

Loading data ...
Data loaded (took 78.52 seconds)


# Run

In [5]:
# --------
# User-guided SLDS parameters
# --------
latent_dim = 2 # number of latent dimensions
# transitions = "standard" # transition class
transitions = "standard"
stride = 10*60 # s
duration = 10*60 # s

length = int(duration/dt)
# start_times = np.arange(0, lfp.shape[0]*dt - duration + 0.1, stride).astype(int)
start_times = np.arange(0, lfp.shape[0]*dt - duration - 0.1, stride).astype(int)
start_times = np.hstack([start_times, lfp.shape[0]*dt - duration - 5])

# areas = ['vlPFC', 'FEF', 'CPB', '7b']
areas = np.unique(electrode_info['area'])
# areas = ['vlPFC']
unit_indices = np.arange(lfp.shape[1])[pd.Series(electrode_info['area']).isin(areas)]
var_names = [f"unit_{unit_num} {electrode_info['area'][unit_num]}" for unit_num in unit_indices]

# --------
# Set the parameters of the SLDS
# --------
emissions_dim = len(unit_indices)     # number of observed dimensions

data_dir = "../../__data__/slds/"
t = time.localtime()
timestamp = time.strftime('%b-%d-%Y_%H%M', t)
data_dir = os.path.join(data_dir, f"slds_big_run_latent_{latent_dim}_duration_{duration}_stride_{stride}_{timestamp}")
os.makedirs(data_dir, exist_ok=True)